<a href="https://colab.research.google.com/github/sutharimanikanta/Golab/blob/main/Brain_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet

Cloning into 'Brain-Tumor-Classification-DataSet'...
remote: Enumerating objects: 3039, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3039 (delta 0), reused 0 (delta 0), pack-reused 3035
Receiving objects: 100% (3039/3039), 79.25 MiB | 30.71 MiB/s, done.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import pickle
import time
import numpy as np
import keras.optimizers
from sklearn.metrics import classification_report

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [ ]:
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import imutils
import cv2
import PIL.Image
import os
%matplotlib inline

demo_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)

In [ ]:
import os

# Define categories
categories = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]

# Define paths for training and testing data
train_dir = "./data/train"
test_dir = "./data/test"

# Function to create directories
def make_directories(base_dir):
    for category in categories:
        os.makedirs(os.path.join(base_dir, category), exist_ok=True)

# Create directories for training data
make_directories(train_dir)

# Create directory for testing data
os.makedirs(test_dir, exist_ok=True)

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.model_selection import train_test_split
def cropAndAugmentation():
    # Augmentation Code
    IMG_SIZE = 80
    ADD_PIXELS = 5  # Adjust as needed
    dim = (IMG_SIZE, IMG_SIZE)
    cwd = os.getcwd()
    directory = ["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
    desired_folder = "Brain-Tumor-Classification-DataSet/Training/"

    # Create copies of the original dataset with different modifications
    datasets = []
    for i in range(3):
        datasets.append([])

    for input_folder in directory:
        input_folder_path = os.path.join(cwd, desired_folder + input_folder)
        if not os.path.exists(input_folder_path):
            raise FileNotFoundError(f"Input folder {input_folder_path} does not exist.")

        for img in os.listdir(input_folder_path):
            image_path = os.path.join(input_folder_path, img)
            image = cv2.imread(image_path)

            # Resize images for the three datasets
            resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            rotated_image = cv2.rotate(resized_image, cv2.ROTATE_90_CLOCKWISE)
            flipped_image = cv2.flip(resized_image, 1)  # Flip along the vertical direction

            # Append resized images to datasets
            datasets[0].append(resized_image)
            datasets[1].append(rotated_image)
            datasets[2].append(flipped_image)

    # Concatenate the three datasets into one
    concatenated_dataset = np.concatenate(datasets, axis=0)

    # Split the dataset into training and testing sets
    X_train, X_test = train_test_split(concatenated_dataset, test_size=0.1, random_state=42)
    # Create an iterator for training data with augmentation
    train_iterator = demo_datagen.flow(X_train, batch_size=32, shuffle=True)

    # Store training and testing data
    data_dir = "./data"
    train_dir = os.path.join(data_dir, "train")
    test_dir = os.path.join(data_dir, "test")

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Store training images
    for idx, img_batch in enumerate(train_iterator):
        for img in img_batch:
            category_folder = directory[idx % len(directory)]
            img_path = os.path.join(train_dir, category_folder, f"image_{idx}.jpg")
            os.makedirs(os.path.join(train_dir, category_folder), exist_ok=True)
            cv2.imwrite(img_path, img)
            idx += 1  # Increment index for the next image

        if idx >= len(X_train):  # Stop when all original training images are processed
            break

    # Store testing images
    for idx, img in enumerate(X_test):
        img_path = os.path.join(test_dir, f"image_{idx}.jpg")
        cv2.imwrite(img_path, img)
    # Count total number of images
    total_images = len(X_train) + len(X_test)
    print("Total number of images:", total_images)
    print("Number of training images:", len(X_train))
    print("Number of testing images:", len(X_test))

# Execute the function
cropAndAugmentation()


Total number of images: 8610
Number of training images: 7749
Number of testing images: 861


In [ ]:
TEST_DIR = '/content/data/test' # test data folder
TRAIN_DIR = '/content/data/train' # train data folder
CATEGORIES =["glioma_tumor", "meningioma_tumor", "no_tumor", "pituitary_tumor"]
from tqdm import tqdm
import random


In [ ]:
IMG_SIZE = 150

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(TRAIN_DIR, category)  # create path
        for img in tqdm(os.listdir(path)):
            # iterate over each image per category
            img_path = os.path.join(path, img)
            img_array = cv2.imread(img_path, cv2.IMREAD_COLOR)  # convert to array
            if img_array is not None:
                # Resize the image to the desired size (e.g., 80x80)
                img_array = cv2.resize(img_array, (80, 80))
                training_data.append(img_array)  # Use only input images
            else:
                print(f"Failed to read image: {img_path}")

    random.shuffle(training_data)
    print("Total number of images in training data:", len(training_data))


100%|██████████| 1937/1937 [00:00<00:00, 11246.29it/s]


In [ ]:
# Convert the list of tuples into a single numpy array
training_data_array = np.array(training_data, dtype=object)

# Save the training data array
np.save('train_data.npy', training_data_array)
print(len(training_data_array))
print("train data")
print()


7749
train data



In [ ]:
training_data_array[5][-1]

3

In [ ]:
# X_train = np.array([i[0] for i in training_data]).reshape(-1,IMG_SIZE,IMG_SIZE,3)
# Y_train = [i[1] for i in training_data]# features

In [ ]:
testing_data = []

def create_testing_data():
    for img in tqdm(os.listdir(TEST_DIR)):
        img_path = os.path.join(TEST_DIR, img)
        if os.path.isfile(img_path):
            img_array = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if img_array is not None:
                # Resize the image to the desired size (e.g., 80x80)
                img_array = cv2.resize(img_array, (80, 80))
                testing_data.append(img_array)
            else:
                print(f"Failed to read image: {img_path}")
        else:
            print(f"Not a file: {img_path}")

    random.shuffle(testing_data)
    print("Total number of images in testing data:", len(testing_data))



100%|██████████| 861/861 [00:00<00:00, 7833.55it/s]

Total number of images in testing data: 861


In [ ]:
# Convert the list of tuples into a single numpy array
testing_data_array = np.array(testing_data, dtype=object)

# Save the training data array
np.save('test_data.npy', training_data_array)
print(len(testing_data))
print("test data")
print()


861
test data



In [ ]:
# Convert to NumPy array
X_train = np.array([i[0] for i in training_data])
Y_train = np.array([i[1] for i in training_data])
# Convert to NumPy array
X_test = np.array([i[0] for i in testing_data])
Y_test = np.array([i[1] for i in testing_data])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time
import numpy as np
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam




In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, Model, optimizers
# import matplotlib.pyplot as plt
# import numpy as np

# class Autoencoder(Model):
#     def __init__(self, latent_dim, shape):
#         super(Autoencoder, self).__init__()  # Call super().__init__()
#         self.latent_dim = latent_dim
#         self.shape = shape
#         self.encoder = tf.keras.Sequential([
#             layers.InputLayer(input_shape=shape),
#             layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.MaxPooling2D((2, 2)),
#             layers.Dropout(0.1),
#             layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.MaxPooling2D((2, 2)),
#             layers.Dropout(0.1),
#             layers.Conv2D(32, (2, 2), activation='relu', padding='same'),
#             layers.MaxPooling2D((2, 2)),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2))  # Adjusted to match the input shape
#         ])
#         self.decoder = tf.keras.Sequential([
#             layers.Conv2DTranspose(32, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(shape[-1], (3, 3), activation='sigmoid', padding='same')
#         ])
#         self.build((None,) + shape)  # Explicitly build the model

#     def call(self, x):
#         encoded = tf.cast(x, tf.float32)  # Cast input data to float32
#         encoded = self.encoder(encoded)
#         decoded = self.decoder(encoded)
#         return decoded

# # Assuming shape is properly defined as (80, 80, 3)
# shape = (80, 80, 3)
# latent_dim = 4
# autoencoder = Autoencoder(latent_dim, shape)



In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class Autoencoder(Model):
    def __init__(self, latent_dim, shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.shape = shape
        self.encoder = tf.keras.Sequential([
            layers.InputLayer(input_shape=shape),
            layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.1),
            layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.1),
            layers.Conv2D(32, (2, 2), activation='relu', padding='same'),
            layers.MaxPooling2D((2, 2)),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2))  # Adjusted to match the input shape
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(32, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
            layers.Dropout(0.1),
            layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
            layers.Dropout(0.1),
            layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
            layers.BatchNormalization(),
            layers.Dropout(0.1),
            layers.Conv2DTranspose(shape[-1], (3, 3), activation='sigmoid', padding='same')
        ])
        self.build((None,) + shape)

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


# Assuming shape is properly defined as (80, 80, 3)
# Assuming shape is properly defined as (80, 80, 3)
shape = (80, 80, 3)  # Update this line with the correct input shape
latent_dim = 4
autoencoder = Autoencoder(latent_dim, shape)


In [ ]:
# import tensorflow as tf

# # Assuming X_train and X_test are properly defined
# X_train_resized = tf.image.resize(X_train, (80, 80))
# X_test_resized = tf.image.resize(X_test, (80, 80))

# # Normalize the pixel values to the range [0, 1]
# X_train_resized = X_train_resized / 255.0
# X_test_resized = X_test_resized / 255.0


In [ ]:
autoencoder.compile(loss='mse',
                          optimizer=Adam(learning_rate=0.1),
                          metrics=['accuracy'],
                          )

In [ ]:
history=autoencoder.fit(X_train, X_train,
                epochs=10,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/tmp/__autograph_generated_file1n8plimu.py", line 10, in tf__call
        encoded = ag__.converted_call(ag__.ld(self).encoder, (ag__.ld(x),), None, fscope)

 

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, Model
# from tensorflow.keras.optimizers import Adam

# class Autoencoder(Model):
#     def __init__(self, latent_dim, shape):
#         super(Autoencoder, self).__init__()  # Call super().__init__()
#         self.latent_dim = latent_dim
#         self.shape = shape
#         self.encoder = tf.keras.Sequential([
#             layers.InputLayer(input_shape=shape),
#             layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2D(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.MaxPooling2D((2, 2)),
#             layers.Dropout(0.1),
#             layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2D(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.MaxPooling2D((2, 2)),
#             layers.Dropout(0.1),
#             layers.Conv2D(32, (2, 2), activation='relu', padding='same'),
#             layers.MaxPooling2D((2, 2)),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2))  # Adjusted to match the input shape
#         ])
#         self.decoder = tf.keras.Sequential([
#             layers.Conv2DTranspose(32, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2DTranspose(64, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.UpSampling2D((2, 2)),  # Adjusted to match the downsampling factor
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Conv2DTranspose(128, (2, 2), activation='relu', padding='same'),
#             layers.BatchNormalization(),
#             layers.Dropout(0.1),
#             layers.Conv2DTranspose(shape[-1], (3, 3), activation='sigmoid', padding='same')
#         ])
#         self.build((None,) + shape)  # Explicitly build the model

#     def call(self, x):
#         encoded = tf.cast(x, tf.float32)  # Cast input data to float32
#         encoded = self.encoder(encoded)
#         decoded = self.decoder(encoded)
#         return decoded

# # Define shape and latent dimension
# shape = (80, 80, 3)
# latent_dim = 4

# # Instantiate autoencoder
# autoencoder = Autoencoder(latent_dim, shape)

# # Compile autoencoder




In [ ]:
# autoencoder.compile(loss='mse',
#                     optimizer=Adam(learning_rate=0.1),
#                     metrics=['accuracy'])

# # Train autoencoder
# history = autoencoder.fit(X_train, X_train,
#                           epochs=5,
#                           shuffle=True,
#                           validation_data=(X_test, X_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

# Define the model architecture using Sequential API
model = Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(80, 80, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.1),

    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),

    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),

    layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Conv2D(8, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),

    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 80, 80, 64)        1792      
                                                                 
 batch_normalization_10 (Ba  (None, 80, 80, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_6 (Conv2D)           (None, 80, 80, 64)        36928     
                                                                 
 batch_normalization_11 (Ba  (None, 80, 80, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 40, 40, 64)        0         
 g2D)                                                            
                                                      

In [ ]:
y_pred = autoencoder.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_classes[1]

27/27 [==============================] - 1s 37ms/step


array([[19, 19, 19],
       [19, 19, 19],
       [20, 20,  0],
       [22, 23,  0],
       [21, 23,  0],
       [21, 23,  0],
       [15, 51,  0],
       [14, 14,  0],
       [12, 12, 74],
       [ 7,  7, 75],
       [ 6,  7, 40],
       [ 5,  6, 35],
       [ 5, 28, 30],
       [ 5, 25, 26],
       [ 6, 22, 22],
       [18, 19, 20],
       [14, 15, 15],
       [12, 13, 14],
       [10, 11, 12],
       [ 8,  9, 10],
       [ 6,  6,  7],
       [ 4,  5,  5],
       [ 4,  4,  4],
       [ 4,  4,  4],
       [ 4,  4,  5],
       [ 4,  4,  4],
       [ 4,  4,  4],
       [ 3,  4,  4],
       [ 3,  3,  4],
       [ 3,  4,  4],
       [ 3,  4,  4],
       [ 4,  4,  4],
       [ 3,  4,  4],
       [ 3,  3,  4],
       [ 2,  3,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  2,  3],
       [ 2,  3,  3],
       [ 2,  3,  3],
       [ 2,  3,  3],
       [ 2,  3,  3],
       [ 2,  3,  3],
       [ 2,  

In [ ]:


# Get the confusion matrix
conf_matrix = confusion_matrix(Y_test, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Compute other evaluation metrics
accuracy = accuracy_score(Y_test, y_pred_classes)
precision = precision_score(Y_test, y_pred_classes, average='macro')
recall = recall_score(Y_test, y_pred_classes, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

27/27 [==============================] - 1s 36ms/step


ValueError: Classification metrics can't handle a mix of binary and unknown targets

In [ ]:
scores = autoencoder.evaluate(X_test, X_test)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

27/27 [==============================] - 1s 38ms/step - loss: 4125.2202 - accuracy: 0.9992
Test loss: 4125.22021484375
Test accuracy: 0.9991746544837952


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Split the encoded test data into training and testing sets for each model
encoded_test_data = autoencoder.encoder(X_test)
X_encoded_train, X_encoded_test, y_encoded_train, y_encoded_test = train_test_split(encoded_test_data, y_test, test_size=0.2, random_state=42)

ResourceExhaustedError: Exception encountered when calling layer 'conv2d_1' (type Conv2D).

{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[861,128,81,81] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D] name: 

Call arguments received by layer 'conv2d_1' (type Conv2D):
  • inputs=tf.Tensor(shape=(861, 80, 80, 128), dtype=float32)

In [ ]:
# Encode the data using the trained autoencoder (remove this part)
# X_encoded_train = autoencoder.encoder.predict(X_train)
# X_encoded_test = autoencoder.encoder.predict(X_test)

# Now you can use the original data for classification

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)
y_pred_knn = knn.predict(X_test)
cm_knn = confusion_matrix(Y_test, y_pred_knn)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, Y_train)
y_pred_rf = rf.predict(X_test)
cm_rf = confusion_matrix(Y_test, y_pred_rf)

# SVM
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, Y_train)
y_pred_svm = svm.predict(X_test)
cm_svm = confusion_matrix(Y_test, y_pred_svm)

# Logistic Regression
lr = LogisticRegression(random_state=42)
lr.fit(X_train, Y_train)
y_pred_lr = lr.predict(X_test)
cm_lr = confusion_matrix(Y_test, y_pred_lr)

# SGD
sgd = SGDClassifier(random_state=42)
sgd.fit(X_train, Y_train)
y_pred_sgd = sgd.predict(X_test)
cm_sgd = confusion_matrix(Y_test, y_pred_sgd)

# MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)
mlp.fit(X_train, Y_train)
y_pred_mlp = mlp.predict(X_test)
cm_mlp = confusion_matrix(Y_test, y_pred_mlp)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, recall_score

# Get reconstructed outputs for X_train and X_test
reconstructed_train = autoencoder.predict(X_train)
reconstructed_test = autoencoder.predict(X_test)

# Assuming y_train and y_test contain the true labels for X_train and X_test respectively
# Compute confusion matrix, F1 score, and recall for train and test sets
y_train_pred = np.argmax(reconstructed_train, axis=3).flatten()
y_train_true = np.argmax(X_train, axis=3).flatten()
train_conf_matrix = confusion_matrix(y_train_true, y_train_pred)
train_f1_score = f1_score(y_train_true, y_train_pred, average='weighted')
train_recall = recall_score(y_train_true, y_train_pred, average='weighted')

y_test_pred = np.argmax(reconstructed_test, axis=3).flatten()
y_test_true = np.argmax(X_test, axis=3).flatten()
test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)
test_f1_score = f1_score(y_test_true, y_test_pred, average='weighted')
test_recall = recall_score(y_test_true, y_test_pred, average='weighted')

print("Train Confusion Matrix:")
print(train_conf_matrix)
print("Train F1 Score:", train_f1_score)
print("Train Recall:", train_recall)

print("\nTest Confusion Matrix:")
print(test_conf_matrix)
print("Test F1 Score:", test_f1_score)
print("Test Recall:", test_recall)


In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_range_test(X_train, X_test, shape, latent_dim, start_lr, end_lr, batch_size, epochs):
    autoencoder = Autoencoder(latent_dim, shape)

    # Define a learning rate scheduler callback
    def lr_scheduler(epoch, lr):
        return start_lr * (end_lr / start_lr) ** (epoch / epochs)

    # Compile the model with an initial learning rate
    autoencoder.compile(loss='mse', optimizer=Adam(learning_rate=start_lr), metrics=['accuracy'])

    # Define the LearningRateScheduler callback
    lr_callback = LearningRateScheduler(lr_scheduler)

    # Train the model with LR range test
    history = autoencoder.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, shuffle=True,
                               validation_data=(X_test, X_test), callbacks=[lr_callback], verbose=0)

    return history.history['loss'], history.history['accuracy'], autoencoder

# Define LR range test parameters
start_lr = 1e-7
end_lr = 10
batch_size = 32
epochs = 10

# Perform LR range test
losses, accuracies, autoencoder = lr_range_test(X_train, X_test, shape, latent_dim, start_lr, end_lr, batch_size, epochs)

# Plot the learning rate vs. loss curve
learning_rates = start_lr * (end_lr / start_lr) ** (np.arange(epochs) / epochs)
plt.plot(learning_rates, losses)
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Loss')
plt.title('Learning Rate vs. Loss')
plt.show()

# Plot the learning rate vs. accuracy curve
plt.plot(learning_rates, accuracies)
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Learning Rate vs. Accuracy')
plt.show()
